In [1]:
import numpy as np
import math
import warnings
import math
import re
import urllib.request
import requests
from bs4 import BeautifulSoup

import tables as tt
import stat_funcs as sf

warnings.filterwarnings('ignore')


In [25]:



class lookup_stats:
    def __init__(self, url):
        self.Dict = {
            'Name' : '',
            'CR' : '',
            'XP' : '',
            'Race' : '',
            'Class' : '',
            'MonsterSource':'',
            'Alignment': '',
            'Size': '',
            'Type': '',
            'SubType': '',
            'Init': '',
            'Senses': '',
            'Aura': '',
            'AC': '',
            'AC_Mods': '',
            'HP': '',
            'HD': '',
            'HP_Mods': '',
            'Saves': '',
            'Fort': '',
            'Ref': '',
            'Will': '',
            'Save_Mods': '',
            'Defensive Abilities': '',
            'DR': '',
            'Immune': '',
            'Resist': '',
            'SR': '',
            'Weaknesses': '',
            'Speed': '',
            'Speed_Mod': '',
            'Melee': '',
            'Ranged': '',
            'Space': '',
            'Reach': '',
            'Special Attacks': '',
            'Spell Like Abilities': '',
            'Spells Known': '',
            'Spells Prepared': '',
            'Spell Domains': '',
            'Abilitiy Scores': '',
            'AbilitiyScore_Mods': '',
            'Base Atk': '',
            'CMB': '',
            'CMD': '',
            'Feats': '',
            'Skills': '',
            'Racial Modifiers': '',
            'Languages': '',
            'SQ': '',
            'Environment': '',
            'Organization': '',
            'Treasure': '',
                    }
        soup = BeautifulSoup(requests.Session().get(url).content,"html5lib")
        self.text = soup.get_text()
        self.Dict['Name'] = re.findall('\n\s*(.*) CR \d*\/?\d*\s*\n', self.text)[0]
        self.relevent_text = re.findall(self.Dict['Name'] + ' CR \d*[\S\s]*', self.text)[0]
        self.relevent_text = self.relevent_text.replace(u"\U00002013", "-")
        self.cat_split = re.split('DEFENSE|OFFENSE|TACTICS|STATISTICS|SPECIAL ABILITIES|ECOLOGY',self.relevent_text)

        
        self.list_of_classes = ['Barbarian', 'Bard', 'Cleric', 'Druid', 'Fighter', 'Monk', 'Paladin', 
                   'Ranger', 'Rogue', 'Sorcerer', 'Wizard', 'Alchemist', 'Cavalier', 'Gunslinger', 
                   'Inquisitor', 'Magus', 'Omdura', 'Oracle', 'Shifter', 'Summoner', 'Witch', 'Vampire Hunter', 'Vigilante',
                    'Arcanist', 'Bloodrager', 'Brawler', 'Hunter', 'Investigator', 'Shaman', 'Skald', 'Slayer',
                    'Swashbuckler', 'Warpriest', 'Kineticist', 'Medium', 'Mesmerist', 'Occultist', 'Psychic', 'Spiritualist',
                    'Antipaladin', 'Ninja', 'Samurai', 'Adept', 'Aristocrat', 'Commoner', 'Expert', 'Warrior']
        
        #for box in range(len(self.cat_split)):
        #    print(box)
        #    print(self.cat_split[box])
        
    def get_top_layer(self):

        self.Dict['CR'], self.Dict['XP'] = re.findall(self.Dict['Name']+'\s*CR\s*(.*)\s*XP (\d*,?\d*)',self.cat_split[0])[0]
        self.Dict['Alignment'], self.Dict['Size'] =  re.findall('([A-Z][A-Z]?) ([a-zA-Z][a-z]+)', self.cat_split[0])[0]
        self.Dict['Type'] = re.findall(self.Dict['Size'] + '\s*([\S\s]*?)(?:\(|Init)', self.cat_split[0])[0]
        self.Dict['SubType'] = re.findall(self.Dict['Type'] + '\s*([\S\s]*?)(?:Init)', self.cat_split[0])[0]
        if 'Aura' in self.cat_split[0]:
            self.Dict['Init'], self.Dict['Senses'], self.Dict['Aura'] = re.findall('Init\s*(.?\d*)\;\s*Senses\s*(.*)\s*Aura\s*(.*)', self.cat_split[0])[0]
        else:
            self.Dict['Init'], self.Dict['Senses'] = re.findall('Init\s*(.?\d*)\;\s*Senses\s*(.*)', self.cat_split[0])[0]
        race_class = re.findall(self.Dict['XP'] + '\s*((?:(?!\s*'+self.Dict['Alignment']+').)*)', self.cat_split[0])[0]
        if race_class.upper().isupper():
            if race_class[-1].isnumeric():
                if len(race_class.split(' ')) ==3:
                    race = race_class.split(' ')[0]
                    cla = race_class.split(' ')[0] + ' ' + race_class.split(' ')[0]
                else:
                    for class_name in self.list_of_classes:
                        if class_name.casefold() in race_class.casefold():
                            race, cla = re.findall('(.*)(' + class_name.casefold() + '.*)',race_class.casefold())[0]
                            break
            else:
                race = race_class
                cla = ''
        else:
            race = ''
            cla = ''
        self.Dict['Race'] = race
        self.Dict['Class'] = cla
    
    def get_defense(self):

        self.Dict['AC'], self.Dict['AC_Mods'] = re.findall('AC (\d*,\s*touch \d*, flat-footed \d*) (\([\S|\s]*[a-z]\))\s*(?:hp|Fort)', self.cat_split[1])[0]
        self.Dict['HP'], self.Dict['HD'] = re.findall(r'\s*hp\s*(\d*)\s*(\([\S|\s]*\))', self.cat_split[1])[0]#self.defense_stats[2]

        self.Dict['Saves'] = re.findall(' (Fort [\S\s]*?)(?:\Z|Def|Imm|Resis|Weak)',self.cat_split[1])[0]
        self.Dict['Fort'] =  re.findall('Fort\s*(.*)',self.Dict['Saves'])[0]
        self.Dict['Ref'] = re.findall('Ref\s*(.*)',self.Dict['Saves'])[0]
        self.Dict['Will'] = re.findall('Will\s*(.*)',self.Dict['Saves'])[0]

        
        if 'Defensive Abilities' in self.cat_split[1]:
            self.Dict['Defensive Abilities'] = re.findall(r'Defensive Abilities ([\S\s]*?)(?:\Z|;)', self.cat_split[1])[0]

        if 'Immune' in self.cat_split[1]:
            self.Dict['Immune'] = re.findall(r'Immune ([\S\s]*?)(?:\Z|;)', self.cat_split[1])[0]
        
        if 'Resist' in self.cat_split[1]:
            self.Dict['Resist'] = re.findall(r'Resist ([\S\s]*?)(?:\Z|[A-Z])', self.cat_split[1])[0]
        
        if 'Weaknesses' in self.cat_split[1]:
            self.Dict['Weaknesses'] = re.findall(r'Weaknesses ([\S\s]*?)(?:\Z|[A-Z])', self.cat_split[1])[0]
    
        if 'DR ' in self.cat_split[1]:
            self.Dict['DR'] = re.findall(r'DR ([\S\s]*?)(?:\Z|;)', self.cat_split[1])[0]

        if 'SR' in self.cat_split[1]:
            self.Dict['SR'] = re.findall(r'SR (\d*)', self.cat_split[1])[0]
    
    def get_offense(self):
        self.Dict['Speed'] = re.findall(r'Speed ((?:(?!Melee|Ranged|Space|Reach|Special Attack|Spell)[\S\s])*)',self.cat_split[2])[0]
        if 'Melee' in self.cat_split[2]:
            self.Dict['Melee'] = re.findall(r'Melee ((?:(?!Ranged|Space|Reach|Special Attack|Spell)[\S\s])*)',self.cat_split[2])[0]
        if 'Ranged' in self.cat_split[2]:
            self.Dict['Ranged'] = re.findall(r'Ranged ((?:(?!Space|Reach|Special Attack|Spell)[\S\s])*)',self.cat_split[2])[0]

        if 'Space' in self.cat_split[2]:
            self.Dict['Space'] = re.findall(r'Space ((?:(?!Reach|Special Attack|Spell)[\S\s])*)',self.cat_split[2])[0]

        if 'Reach' in self.cat_split[2]:
            self.Dict['Reach'] = re.findall(r'Reach ((?:(?!Special Attacks|Spell)[\S\s])*)',self.cat_split[2])[0]
            
        if 'Special Attacks' in self.cat_split[2]:
            self.Dict['SpecialAttacks'] = re.findall(r'Special Attacks ((?:(?!Spell)[\S\s])*)',self.cat_split[2])[0]

        if 'Spell-Like' in self.cat_split[2]:
            if 'Spells Known' in self.cat_split[2]:
                self.Dict['Spell Like Abilities'] = re.findall(r'Spell\WLike Abilities ((?:(?!([A-Z]\w*)?\s*Spells)[\S\s])*)',self.cat_split[2])[0]
                if len(self.Dict['Spell Like Abilities']) > 1:
                    self.Dict['Spell Like Abilities'] = self.Dict['Spell Like Abilities'][0]
            else:
                self.Dict['Spell Like Abilities'] = re.findall(r'Spell\WLike Abilities (.*)',self.cat_split[2])[0]

        if 'Spells Known' in self.cat_split[2]:
            self.Dict['Spells Known'] = re.findall(r'Spells Known ([\S\s]*)',self.cat_split[2])[0]
        
    def get_statistics(self):
        
        
        #Sometimes there is a TACTICS section before the STATISTICS section. 
        #This effects the index of cat_split that should be used
        for i in range(len(self.cat_split)):
            find_section = re.findall(r'(Str[\S|\s]*Cha\s\d*)',self.cat_split[i])
            if len(find_section) > 0:
                sec=i
                break
        self.Dict['Abilitiy Scores'] = re.findall(r'(Str[\S|\s]*Cha\s\d*)',self.cat_split[sec])[0]
        self.Dict['Base Atk'] = re.findall(r'Base\s*Atk\s(\W?\d*)', self.cat_split[sec])[0]
        self.Dict['CMB'] = re.findall(r'CMB([\S|\s]*)CMD\s*\d*\s*', self.cat_split[sec])[0]
        self.Dict['CMD'] = re.findall(r'CMD\s((?:(?!\s*[A-Z]).)*)', self.cat_split[sec])[0]
        if 'Feats' in self.cat_split[sec]:
            self.Dict['Feats'] = re.findall(r'Feats\s((?:(?!\s*Skills|Racial|Languages|SQ).)*)', self.cat_split[sec])[0]

        if 'Skills' in self.cat_split[sec]:
            self.Dict['Skills'] = re.findall(r'Skills\s((?:(?!\s*Racial|Languages|SQ).)*)', self.cat_split[sec])[0]
        if 'Racial Mods' in self.cat_split[sec]:
            self.Dict['Racial Modifiers'] = re.findall(r'Racial\sModifiers\s((?:(?!\s*Languages|SQ).)*)', self.cat_split[sec])[0]
        if 'Languages' in self.cat_split[sec]:
            self.Dict['Languages'] = re.findall(r'Languages\s((?:(?!\s*SQ).)*)', self.cat_split[sec])[0]
        if ' SQ ' in self.cat_split[sec]:
            self.Dict['SQ'] = re.findall(r'\sSQ\s(.*)', self.cat_split[sec])[0]
        

        
    def _clean_it_up(self):
        for key in self.Dict:
            if '\n' in self.Dict[key]:
                self.Dict[key] = self.Dict[key].replace('\n', ' ')
            if len(self.Dict[key]) > 0:
                if self.Dict[key][0] == ' ':
                    self.Dict[key] = self.Dict[key][1:]
                if self.Dict[key][-1] == ' ':
                    self.Dict[key] = self.Dict[key][:-1]
        
            
    def get_em_all(self):
        self.get_top_layer()
        self.get_defense()
        self.get_offense()
        self.get_statistics()
        self._clean_it_up()
        return self.Dict

        
    


In [32]:




class Apply_Skeleton_Template:
    def __init__(self,data):
        self.data = data
        
        self.Skel_nat_ac = {
                    'Fine':0,
                    'Diminutive':0,
                    'Tiny':0,
                    'Small':1,
                    'Medium':2,
                    'Large':2,
                    'Huge':3,
                    'Gargantuan':6,
                    'Colossal':10,
                            }

        self.Skel_claw_dam = {
                    'Fine':'-',
                    'Diminutive':'1',
                    'Tiny':'1d2',
                    'Small':'1d3',
                    'Medium':'1d4',
                    'Large':'1d6',
                    'Huge':'1d8',
                    'Gargantuan':'2d6',
                    'Colossal':'2d8',
                                }
        self.Skel_CR_table = {
                    '1/2': '0.17',
                    '0.5': '0.17',
                    '1':'0.33',
                    '2':'1',
                    '3':'1',
                    '4':'2',
                    '5':'2',
                    '6':'3',
                    '7':'3',
                    '8':'4',
                    '9':'4',
                    '10':'5',
                    '11':'5',
                    '12':'6',
                    '13':'6',
                    '14':'6',
                    '15':'7',
                    '16':'7',
                    '17':'7',
                    '18':'8',
                    '19':'8',
                    '20':'8'
                            }
        self.subtypes_to_remove = ['good', 'evil', 'lawful', 'chaotic', 'giant', 'dwarf', 'human', 'elf', 
                   'gnome', 'orc', 'catfolk', 'goblin','goblinoid','halfling','kitsune', 'ratfolk']
        
    def _skeleton_HD(self, total_hit_dice):
        class_list = list(filter(None, re.split('(\d)|(\/)',str(self.data['Class']))))
        levels = [int(i) for i in class_list if i.isnumeric()]
        for lvl in levels:
            total_hit_dice -= lvl
        if total_hit_dice > 0 and total_hit_dice < 1.0:
            total_hit_dice = str(0.5)
        else:
            total_hit_dice = int(max(1, total_hit_dice))
        return str(total_hit_dice)
    
    def _skelton_AC(self, stats):
        total_AC = 10
        nat = self.Skel_nat_ac[self.data['Size']]
        dex = math.floor((float(stats[1][1])-10)/2)
        size = int(tt.size_bonus[self.data['Size']])
        total_AC += nat
        total_AC += dex
        total_AC += size
        ff = 10 + nat + size
        if dex < 0:
            ff += dex
        touch = 10 + dex + size
        mods = '('
        if dex > 0:
            mods += '+' + str(dex) + ' Dex'
        elif dex < 0:
            mods += str(dex) + ' Dex'
        if nat > 0:
            mods += ', +' + str(nat) + ' natural'
        elif nat < 0:
            mods += ', ' + str(nat) + ' natural'
        if size > 0:
            mods += ', +' + str(size) + ' size'
        elif size < 0:
            mods += ', ' + str(size) + ' size'
        mods += ')'
        if dex == 0 and nat ==0:
            mods =''
        AC_all = str(total_AC) + ', touch ' + str(touch) + ', flat-footed ' + str(ff)

        return mods, AC_all

    def _get_skelly_saves(self, total_hit_dice, stats):
        saves = sf.get_saves(self.data)
        saves[0][1] = str(math.floor(total_hit_dice/3) + math.floor((float(stats[5][1])-10)/2))
        saves[1][1] = str(math.floor(total_hit_dice/3) + math.floor((float(stats[1][1])-10)/2))
        saves[2][1] = str(math.floor(total_hit_dice/2)+2 + math.floor((float(stats[4][1])-10)/2))
        saves = ', '.join([' '.join(save) for save in saves])
        return saves

    def _get_skelly_stats(self, stats):
        stats[1][1] = str(int(stats[1][1])+2)
        stats[2][1] = '-'
        stats[3][1] = '-'
        stats[4][1] = '10'
        stats[5][1] = '10'
        stats_str = ', '.join([' '.join(stat) for stat in stats])
        return stats, stats_str
    
    def _get_skelly_ranged(self, old_bab, stats):
        highest_ab = int(self.data['Base Atk'])+math.floor((float(stats[1][1])-10)/2) + int(tt.size_bonus[self.data['Size']])
        if math.floor((float(stats[0][1])-10)/2) >= 0:
            pm = '+'
        else:
            pm =''
        
        ranged = sf.get_attacks(self.data, me_ra='Ranged')
        new_ranged = []
        for fr in range(len(ranged)):
            for at in range(len(ranged[fr])):
                if ranged[fr][at][0][0].isalpha():
                    wep = ranged[fr][at][0]
                else:
                    wep = ranged[fr][at][1]
                iterative_bonus = int(self.data['Base Atk'])
                na = 0
                full_attack_bonus = ''
                while iterative_bonus >= 0:
                    bonus = highest_ab - na*5
                    if bonus >= 0:
                        bonus = '+' + str(bonus)
                    else:
                        if str(bonus)[0] != '-':
                            bonus = '-' + str(bonus)
                        else:
                            bonus = str(bonus)
                    full_attack_bonus += bonus
                    if iterative_bonus - 5>= 0:
                        full_attack_bonus += '/'
                    na += 1
                    iterative_bonus -= 5
                wep_damage = re.findall('(\d*d\d*)[+-]', ranged[fr][at][-1][1:])[0]
                try:
                    crit=tt.weapons_with_crit[wep.lower()]
                except:
                    crit=''
                if 'composite' in wep:
                    extra_damage = math.floor((float(stats[0][1])-10)/2)
                else:
                    extra_damage = 0
                if math.floor(extra_damage) >= 0:
                    extra_damage = '+' + str(extra_damage)
                full_damage = wep_damage+str(extra_damage)
                new_ranged.append([' '.join([wep, full_attack_bonus, '(' + full_damage + ' ' +crit+')'])])
        new_ranged = [', '.join(i) for i in new_ranged]
        new_ranged = ' or '.join(new_ranged)
        return new_ranged
        
    
    def _get_skelly_melee(self, old_bab, stats):
        highest_ab = int(self.data['Base Atk'])+math.floor((float(stats[0][1])-10)/2) + int(tt.size_bonus[self.data['Size']])
        damage = self.Skel_claw_dam[self.data['Size']]
        if math.floor((float(stats[0][1])-10)/2) >= 0:
            pm = '+'
        else:
            pm =''
        if highest_ab >= 0:
            claw_attack = '2 claws +' + str(highest_ab) + ' (' + damage + pm + str(math.floor((float(stats[0][1])-10)/2)) + ')'
        else:
            claw_attack = '2 claws ' + str(highest_ab) + ' (' + damage + pm + str(math.floor((float(stats[0][1])-10)/2)) + ')'
    
        melee = sf.get_attacks(self.data, me_ra='Melee')
        if melee == [[]]:
            new_melee = claw_attack
        else:
            new_melee = []
            check_claw_count=0
            nat_full = []
            for fr in range(len(melee)):
                for at in range(len(melee[fr])):
                    if melee[fr][at][0][0].isalpha():
                        wep = melee[fr][at][0]
                    else:
                        wep = melee[fr][at][1]
                    if wep not in tt.nat_attack_list:
                        iterative_bonus = int(self.data['Base Atk'])
                        na = 0
                        full_attack_bonus = ''
                        while iterative_bonus >= 0:
                            bonus = highest_ab - na*5
                            if bonus >= 0:
                                bonus = '+' + str(bonus)
                            else:
                                if str(bonus)[0] != '-':
                                    bonus = '-' + str(bonus)
                                else:
                                    bonus = str(bonus)
                            full_attack_bonus += bonus
                            if iterative_bonus - 5>= 0:
                                full_attack_bonus += '/'
                            na += 1
                            iterative_bonus -= 5
                        wep_damage = re.findall('(\d*d\d*)[+-]', melee[fr][at][-1][1:])[0]
                        try:
                            crit=tt.weapons_with_crit[wep.lower()]
                        except:
                            crit=''
                        extra_damage = math.floor((float(stats[0][1])-10)/2)
                        if math.floor(extra_damage) >= 0:
                            extra_damage = '+' + str(extra_damage)
                        full_damage = wep_damage+str(extra_damage)
                        new_melee.append([' '.join([wep, full_attack_bonus, '(' + full_damage + ' ' +crit+ ')'])])
                    else:
                        if melee[fr][at][0][0].isnumeric():
                            wep = melee[fr][at][0][0] + ' ' + wep
                        bonus = highest_ab
                        if bonus >= 0:
                            bonus = '+' + str(bonus)
                        else:
                            if str(bonus)[0] != '-':
                                bonus = '-' + str(bonus)
                            else:
                                bonus = str(bonus)
                        #wep_damage = re.findall('(.*)[+-]', melee[fr][at][-1][1:])[0]
                        wep_damage = re.findall('(\d*d\d*)[+-]', melee[fr][at][-1][1:])[0]
                        extra_damage = math.floor((float(stats[0][1])-10)/2)
                        if math.floor(extra_damage) >= 0:
                            extra_damage = '+' + str(extra_damage)
                        full_damage = wep_damage+str(extra_damage)
                        nat_full.append(' '.join([wep, bonus, '(' + full_damage + ')']))
                        #if wep == 'claw' or wep == 'claws':
                        if 'claw' in wep:
                            check_claw_count+=1
            if check_claw_count == 0:
                nat_full.append(claw_attack)
            new_melee.append(nat_full)
            new_melee = [', '.join(i) for i in new_melee]
            #print(new_melee)
            new_melee = ' or '.join(new_melee)
        return new_melee
    
    def skeleton_temp(self,remove_fly=True):
        total_hit_dice, hit_dice = sf.get_hit_dice(self.data)
        if float(total_hit_dice) > 20:
            print('Only creatures with 20 or fewer HD can be made into skeletons!')
            print('This one has ', hit_dice)
            return 0
        #if data['Type'] == 'undead':
        #    print('Creature is already undead!!!')
        #    return 0
        self.data['Name'] += ' Skeleton'
        self.data['Type'] = 'undead'
        #if data['SubType'] != 0:
        for subtype in self.subtypes_to_remove:
            self.data['SubType'].replace(subtype,'')
        
        total_hit_dice = self._skeleton_HD(total_hit_dice)
        self.data['HD'] = str(total_hit_dice) + 'd8'
        self.data['HP'] = str(math.floor(float(total_hit_dice) * 4.5))
        self.data['CR'] = self.Skel_CR_table[str(total_hit_dice)]
        self.data['XP'] = tt.exp_by_CR[self.data['CR']]
        
        if 'fly' in self.data['Speed']:
            if remove_fly:
                fly_speed = re.findall('(fly.*\(?[a-zA-Z]*\);?,?\s*)', self.data['Speed'])[0]
                self.data['Speed'] = ' '.join(re.findall('([\S\s]*)fly.*\(?[a-zA-Z]*\);?,?\s*([\S\s]*)', self.data['Speed'])[0])
                print('Fly speed '+fly_speed+' removed!')
                print('If this flight is from a magical source that doesn\'t depend in class levels/body features, rerun with remove_fly=False')
        if 'weapon training' in self.data['SQ']:
            weapon_training = re.findall('weapon training\s*(\d)')
        self.data['Alignment'] = 'NE'
        stats = sf.get_stats(self.data)
        #print(stats)
        stats, stats_str = self._get_skelly_stats(stats)
        #print(stats)
        self.data['Abilitiy Scores'] = stats_str
        #print(stats)
        old_bab = float(self.data['Base Atk'])
        self.data['Base Atk'] = str(math.floor(int(total_hit_dice)*0.75))
        self.data['Skills'] = ''
        self.data['Defensive Abilities'] = ''
        self.data['DR'] = '5/bludgeoning'
        self.data['Immune'] = 'cold, undead traits'
        self.data['Saves'] = self._get_skelly_saves(float(total_hit_dice), stats)
        #self.data['Ref'] = math.floor(float(hit_dice[0])/3) + math.floor((float(data['Dex'])-10)/2)
        #self.data['Fort'] = math.floor(float(hit_dice[0])/3) + math.floor((float(data['Cha'])-10)/2)
        #self.data['Will'] = math.floor(float(hit_dice[0])/2)+2 + math.floor((float(data['Wis'])-10)/2)
        self.data['Feats'] = 'Improved Initiative'
        self.data['Melee'] = self._get_skelly_melee(old_bab, stats)
        if self.data['Ranged'] != '':
            self.data['Ranged'] = self._get_skelly_ranged(old_bab, stats)
    
        AC_mods, AC_all = self._skelton_AC(stats)
        self.data['AC_Mods'] = AC_mods
        #self.data['AC'] = AC_split[0] + ', ' +  AC_split[1] + ' ' + AC_split[2] + ', ' + AC_split[3] + ' ' + AC_split[4]
        self.data['AC'] = AC_all
        self.data['Init'] = str(math.floor((float(stats[1][1])-10)/2) + 4)
        self.data['Environment'] = 'Any, usually the same as the base creature'
        self.data['SQ']=''
        self.data['CMB'] = str(math.floor((float(stats[0][1])-10)/2) + math.floor(int(total_hit_dice)*0.75) - int(tt.size_bonus[self.data['Size']]))
        self.data['CMD'] = str(10 + math.floor((float(stats[0][1])-10)/2) + math.floor((float(stats[1][1])-10)/2) + math.floor(int(total_hit_dice)*0.75) - int(tt.size_bonus[self.data['Size']]))
        self.data['Aura'] = ''
        self.data['Spell Like Abilities'] = ''
        self.data['Spells Known'] = ''
        self.data['Spells Prepared'] = ''
        self.data['Spell Domains'] = ''
        self.data['Class'] = ''
    
        return self.data


In [34]:
#url = 'https://www.d20pfsrd.com/bestiary/unique-monsters/cr-11/graveknight/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/oozes/jelly-ochre/'
#url = 'https://www.d20pfsrd.com/bestiary/monster-listings/dragons/dragon/metallic-copper/adult-copper-dragon/'
url = 'https://www.d20pfsrd.com/bestiary/monster-listings/animals/felines/cat/'
#url = 'https://www.d20pfsrd.com/bestiary/npc-s/npcs-cr-8/cultist-human-adept-10/'
stat_block = lookup_stats(url)
data_for_mon = stat_block.get_em_all()
skel_to_be = Apply_Skeleton_Template(data_for_mon)
data_for_skel = skel_to_be.skeleton_temp()
#print(data_for_skel)
sf.print_it_out(data_for_skel)

Cat Skeleton	CR 0.33
XP	135
 
NE     Tiny     undead 
Init 7; Senses low-light vision, scent; Perception +5

--------------------------------------------------------------
DEFENSE
--------------------------------------------------------------
AC 15, touch 15, flat-footed 12 (+3 Dex, +2 size)
hp 4 (1d8)
Fort 0, Ref 3, Will 2  
DR 5/bludgeoning 
Immune cold, undead traits 

--------------------------------------------------------------
OFFENSE
--------------------------------------------------------------
Speed 30 ft.
Melee 2 claws -2 (1d2-4), bite -2 (1d3-4) 
Space 2-1/2 ft.; 
Reach 0 ft.   

--------------------------------------------------------------
STATISTICS
--------------------------------------------------------------
Str 3, Dex 17, Con -, Int -, Wis 10, Cha 10
Base Atk 0 
CMB -6 
CMD 7 
Feats Improved Initiative 

